![Py4Eng](img/logo.png)

# Memory model
## Yoav Ram

# Python's memory model

Usually people talk about *call-by-reference* and *call-by-value* and stuff like that. 
We'll try to keep away from that, so that we don't confuse things with previous notions.

We'll follow some of the stuff written by Jeff Knupp in a post called [Is Python call-by-value or call-by-reference? Neither](http://www.jeffknupp.com/blog/2012/11/13/is-python-callbyvalue-or-callbyreference-neither/) and some of the stuff from Python's [official docs](https://docs.python.org/3.5/reference/datamodel.html).

## C++ example

Think about what happens when the following C++ code is executed:

```c
string some_guy = "Fred";
// ...
some_guy = "George";
```

The variable `some_guy` refers to a location in memory, and the value 'Fred' is inserted in that location. 

Later, the contents of the memory location referred to by `some_guy` are changed to 'George'. 
The previous value no longer exists; it was overwritten. 

## What happens in Python?

In Python, [_(almost) everything is an object_](https://docs.python.org/3.5/reference/datamodel.html):

> Objects are Python’s abstraction for data. All data in a Python program is represented by objects or by relations between objects.

 What we commonly refer to as _variables_ in Python are more properly called _names_. 
 Likewise, _assignment_ is really the _binding_ of a _name_ to an _object_. 
 Each _binding_ has a scope that defines its visibility, usually the block in which the name originates.

In [5]:
some_guy = 'Fred'
# ...
some_guy = 'George'

On line 1, we create a _binding_ between a _name_, `some_guy`, and a string _object_ containing 'Fred'. 
In the context of program execution, the _environment_ is altered; a _binding_ of the name `some_guy` to a string _object_ is created in the _scope_ of the block where the statement occurred. 
When we later say `some_guy = 'George'`, the string _object_ containing 'Fred' is unaffected. 
We've just changed the _binding_ of the name `some_guy`. 
We haven't, however, changed either the 'Fred' or 'George' string _objects_. 
As far as we're concerned, they may live on indefinitely (see more below).

## A more complex example

In [7]:
some_guy = 'Fred'

first_names = []
first_names.append(some_guy)
print(some_guy is first_names[0])

another_list_of_names = first_names
another_list_of_names.append('George')
some_guy = 'Bill'

#print(some_guy, first_names, another_list_of_names) # what would this give?

True


So what would be printed in the final line? 

Line 1: the _binding_ of `some_guy` to the string _object_ containing 'Fred' is added to the block's _namespace_. 

Line 2: the name `first_names` is bound to an empty list _object_. 

Line 4: a method is called on the list _object_ `first_names` is bound to, appending the _object_ `some_guy` is bound to.
At this point, there are still only two _objects_ that exist: the string _object_ and the list _object_. 
`some_guy` and `first_names[0]` both refer to the same object (indeed, line 5 shows this).

Line 7: a new _name_ is bound: `another_list_of_names`. 
Assignment between _names_ does not _create_ a new _object_. 
Rather, both _names_ are simply bound to the same _object_. 
As a result, the string _object_ and list _object_ are still the only _objects_ that have been created by the interpreter.

Line 8: a member function is called on the _object_ `another_list_of_names` is bound to and it is **mutated** to contain a _reference_ to a new _object_: 'George'.

Line 9: the _name_ `some_guy` is bound to a new string _object_, 'Bill'.

So the output of the code is:

In [8]:
print(some_guy, first_names, another_list_of_names)

Bill ['Fred', 'George'] ['Fred', 'George']


### Important point 

There are **two kinds of objects in Python**:

- A **mutable** object exhibits time-varying behavior. Changes to a mutable object are visible through all names bound to it. Python's lists and dictionaries are examples of mutable objects. 
- An **immutable** object does not exhibit time-varying behavior. The value of immutable objects can not be modified after they are created. They can be used to compute the values of new objects.

This dichotomy is necessary because, again, _everything in Python is an object_. 

## Functions

Calling `foo(x)` creats a binding within the scope of `foo` of the name `x` to the object bounded to the function argument at the time the function is called. 

If `x` is bounded to a **mutable** object and `foo` changes its value, then these **changes will be have an effect outside the function scope**:

In [9]:
def foo(x):
    x.append(42)
    print('inside:', x)

answer = []
foo(answer)
print('outside:', answer)

inside: [42]
outside: [42]


On the other hand, if `x` refers to an **immutable** object, the most that `foo` can do is bind `x` in its local scope to some other object, and **changes will not have an effect outside function scope**:

In [10]:
def foo(x):
    x = 'new value'
    print('inside:', x)

answer = 'old value'
foo(answer)
print('outside:', answer)

inside: new value
outside: old value


## Identity

Everything is an object and every object has an **identity** (and also a **type**, and a **value**)

An object’s identity *never changes once it has been created*; you may *think of it* as the object’s address in memory. 

The `is` operator compares the identity of two objects; the `id()` function returns an integer representing its identity.

**Note**: In *CPython*, `id(x)` is the memory address where `x` is stored.

Some examples:

In [13]:
x = 5
print(id(x))
x = 6
print(id(x))

1481174928
1481174960


In [14]:
y = x
print(id(x), id(y))
print(x == y)
print(x is y)

1481174960 1481174960
True
True


In [18]:
x = 5
y = 5
print(id(x), id(y))
print(x == y)
print(x is y)

1481174928 1481174928
True
True


In [19]:
x = 5
y = 1
print(id(x), id(y))
print(x == y)
print(x is y)

1481174928 1481174800
False
False


In [20]:
x = "Hello"
y = "Hello"
print(x == y)
print(x is y)

True
True


In [21]:
x = [1, 2, 3]
y = [1, 2, 3]
print(x == y)
print(x is y)

True
False


## Live on indefinitely?

Objects are **never explicitly destroyed**; however, when they become unreachable they may be **garbage-collected**. 
A Python implementation is allowed to postpone garbage collection or omit it altogether — it is a matter of implementation quality how garbage collection is implemented, as long as no objects are collected that are still reachable.

_CPython_ uses a reference-counting scheme with (optional) delayed detection of cyclically linked garbage, which collects most objects **as soon as they become unreachable**, but is not guaranteed to collect garbage containing circular references.

See the documentation of the `gc` module for information on controlling the collection of cyclic garbage. 
Other implementations act differently and CPython may change.
Do not depend on immediate finalization of objects when they become unreachable (so you should always **close files explicitly** or open them inside a context manager).

# References

-  [Is Python call-by-value or call-by-reference? Neither](http://www.jeffknupp.com/blog/2012/11/13/is-python-callbyvalue-or-callbyreference-neither/) by Jeff Knupp
- [Drastically Improve Your Python: Understanding Python's Execution Model](http://www.jeffknupp.com/blog/2013/02/14/drastically-improve-your-python-understanding-pythons-execution-model/) by Jeff Knupp

## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Python for Engineers_](https://github.com/yoavram/Py4Eng) course.

The notebook was written using [Python](http://python.org/) 3.7.
Dependencies listed in [environment.yml](../environment.yml), full versions in [environment_full.yml](../environment_full.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)